# Construct pseudo-OOD data from a single image


In [1]:

import torch
import torch.nn as nn
from torchvision.models import *


In [2]:
# create pesudo OOD data
import os
import torch
import torchvision.transforms as transforms
from PIL import Image
import random

def split_image_to_patches(image, patch_size=64):
    _, height, width = image.size()
    num_patches_h = height // patch_size
    num_patches_w = width // patch_size
    patches = []
    for h in range(num_patches_h):
        for w in range(num_patches_w):
            start_h, start_w = h * patch_size, w * patch_size
            end_h, end_w = start_h + patch_size, start_w + patch_size
            patch = image[:, start_h:end_h, start_w:end_w]
            patches.append(patch)
    return patches

def shuffle_patches(patches):
    shuffled_patches = patches.copy()
    random.shuffle(shuffled_patches)
    return shuffled_patches

def combine_patches(patches, image_size):
    num_patches = len(patches)
    num_patches_per_row = image_size // patches[0].shape[1]
    combined_image = torch.cat([torch.cat(patches[i:i+num_patches_per_row], dim=2) for i in range(0, num_patches, num_patches_per_row)], dim=1)
    return combined_image

def process_image(image_path, output_folder, image_size=256, patch_size=64):
    # 加载图片并进行处理
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
    ])
    image = transform(image).unsqueeze(0)[0]
    # print(image.size())

    # 划分不重叠的patch并进行shuffle
    patches = split_image_to_patches(image, patch_size=patch_size)
    shuffled_patches = shuffle_patches(patches)

    # 组合patch得到新的图片
    shuffled_image = combine_patches(shuffled_patches, image_size)

    # 输出处理后的图片
    # print("Processed image shape:", shuffled_image.shape)

    # 保存处理后的图片
    _, image_name = os.path.split(image_path)

    # output_path = os.path.join(output_folder, f"processed_{image_name}")
    output_path = os.path.join(output_folder, f"{image_name}")

    transforms.ToPILImage()(shuffled_image.squeeze(0)).save(output_path)

def main():
    # 假设有一个包含图像数据的文件夹

    # # CIFAR100
    # input_folder_root = "data/cifar100_images/train"
    # output_folder_root = "data/cifar100_images/train_OOD_28"

    # # CIFAR100 few-shot
    # input_folder_root = "data/cifar100_few_shot_16/train"
    # output_folder_root = "data/cifar100_few_shot_16/train_OOD_28"

    # # ImageNet-R
    # input_folder_root = os.path.join('/home/zhangwentao/Dataset2', 'imagenet_r', 'train')
    # output_folder_root = "data/imagenet_r/train_OOD_28"

    # Skin40
    # input_folder_root = '/home/zhangwentao/Datasets2/SD-198/images'
    # output_folder_root = "data/skin40_OOD_28"
    # input_folder_root = '/home/2021/wentao/Datasets/SD-198/images'
    # output_folder_root = '/home/2021/wentao/Datasets/skin40_OOD_56'
    
    # CUB200
    input_folder_root = "/home/2021/wentao/Datasets/CUB_200_2011/images"
    output_folder_root = "/home/2021/wentao/Datasets/CUB_200_2011/images_OOD_28"

    patch_size = 28
    image_size = 224
    path_file = os.listdir(input_folder_root)

    cnt = 0
    
    for pth_file in path_file:
        input_folder = os.path.join(input_folder_root, pth_file)
        # output_folder = os.path.join(output_folder_root, "ood_"+ pth_file + "_"+ str(patch_size))
        output_folder = os.path.join(output_folder_root,  pth_file) 

        # 确保输出文件夹存在
        os.makedirs(output_folder, exist_ok=True)

        # 遍历文件夹中的图像文件并进行处理
        image_files = [file for file in os.listdir(input_folder) if file.endswith(".jpg") or file.endswith(".png") or file.endswith(".JPEG")]
        for image_file in image_files:
            image_path = os.path.join(input_folder, image_file)
            process_image(image_path, output_folder, image_size=image_size, patch_size=patch_size)
            cnt += 1
    print("A total of {} pieces of pseudo OOD data were generated.".format(cnt))

if __name__ == "__main__":
    main()

A total of 11788 pieces of pseudo OOD data were generated.


# Construct pseudo-OOD data from random two images

## create pesudo OOD data from single file

In [ ]:

import os
import torch
import torchvision.transforms as transforms
from PIL import Image
import random
import torchvision

def split_image_to_patches(image, patch_size=64):
    _, height, width = image.size()
    num_patches_h = height // patch_size
    num_patches_w = width // patch_size
    patches = []
    for h in range(num_patches_h):
        for w in range(num_patches_w):
            start_h, start_w = h * patch_size, w * patch_size
            end_h, end_w = start_h + patch_size, start_w + patch_size
            patch = image[:, start_h:end_h, start_w:end_w]
            patches.append(patch)
    return patches

def shuffle_patches(patches):
    shuffled_patches = patches.copy()
    random.shuffle(shuffled_patches)
    return shuffled_patches

def combine_patches(patches, image_size):
    num_patches = len(patches)
    num_patches_per_row = image_size // patches[0].shape[1]
    combined_image = torch.cat([torch.cat(patches[i:i+num_patches_per_row], dim=2) for i in range(0, num_patches, num_patches_per_row)], dim=1)
    return combined_image

def swap_patches(image1, image2, swap_ratio=0.4):
    num_patches = len(image1)
    num_swaps = int(num_patches * swap_ratio)
    swap_indices = random.sample(range(num_patches), num_swaps)
    for idx in swap_indices:
        image1[idx], image2[idx] = image2[idx], image1[idx]
    return image1, image2



def precess_image_two(input_folder, output_folder, image_size=256, patch_size=64, swap_ratio=0.4):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
    ])
    
    
    image_files = [file for file in os.listdir(input_folder) if file.endswith(".jpg") or file.endswith(".png") or file.endswith(".JPEG")]
    random.shuffle(image_files)
    # print(image_files)

    if len(image_files) < 2:
        print("At least 2 image files needed in the input folder.")
        return
    if len(image_files)%2 == 0:
        total_images = len(image_files)
        flag = False
    else:
        total_images = len(image_files) - 1
        flag = True

    for idx in range(0, total_images, 2):
        img_path_1 = os.path.join(input_folder, image_files[idx])
        image_1 = Image.open(img_path_1).convert("RGB")
        image_1 = transform(image_1).unsqueeze(0)[0]
        patches_1 = split_image_to_patches(image_1, patch_size=patch_size)
        shuffled_patches_1 = shuffle_patches(patches_1)

        img_path_2 = os.path.join(input_folder, image_files[idx+1])
        image_2 = Image.open(img_path_2).convert("RGB")
        image_2 = transform(image_2).unsqueeze(0)[0]
        patches_2 = split_image_to_patches(image_2, patch_size=patch_size)
        shuffled_patches_2 = shuffle_patches(patches_2)

        shuffled_patches_1, shuffled_patches_2 = swap_patches(shuffled_patches_1, shuffled_patches_2, swap_ratio=swap_ratio)

        combined_image1 = combine_patches(shuffled_patches_1, image_size)
        combined_image2 = combine_patches(shuffled_patches_2, image_size)

        torchvision.utils.save_image(combined_image1, os.path.join(output_folder, "combined_" + image_files[idx].strip(".JPEG") + "_" + image_files[idx+1].strip(".JPEG") +"_image1.jpg"))
        torchvision.utils.save_image(combined_image2, os.path.join(output_folder, "combined_" + image_files[idx].strip(".JPEG") + "_" + image_files[idx+1].strip(".JPEG") +"_image2.jpg"))
    # print(idx)
    if flag:
        img_path_1 = os.path.join(input_folder, image_files[idx+1])
        image_1 = Image.open(img_path_1).convert("RGB")
        image_1 = transform(image_1).unsqueeze(0)[0]
        patches_1 = split_image_to_patches(image_1, patch_size=patch_size)
        shuffled_patches_1 = shuffle_patches(patches_1)

        img_path_2 = os.path.join(input_folder, image_files[idx+2])
        image_2 = Image.open(img_path_2).convert("RGB")
        image_2 = transform(image_2).unsqueeze(0)[0]
        patches_2 = split_image_to_patches(image_2, patch_size=patch_size)
        shuffled_patches_2 = shuffle_patches(patches_2)

        shuffled_patches_1, shuffled_patches_2 = swap_patches(shuffled_patches_1, shuffled_patches_2, swap_ratio=swap_ratio)

        combined_image1 = combine_patches(shuffled_patches_1, image_size)
        combined_image2 = combine_patches(shuffled_patches_2, image_size)

        torchvision.utils.save_image(combined_image1, os.path.join(output_folder, "combined_" + image_files[idx+1].strip(".JPEG") + "_" + image_files[idx+2].strip(".JPEG") +"_image1.jpg"))
        torchvision.utils.save_image(combined_image2, os.path.join(output_folder, "combined_" + image_files[idx+1].strip(".JPEG") + "_" + image_files[idx+2].strip(".JPEG") +"_image2.jpg"))
    # print("Finish!")


def main():
    # 假设有一个包含图像数据的文件夹
    input_folder_root = "/data31/data31_3/Public/Datasets/imagenet_few/val_1"  # 输入文件夹路径
    output_folder_root = "/data31/data31_3/Public/Datasets/imagenet_few/output" # 输出文件夹路径
    image_size = 224
    patch_size = 32
    swap_ratio = 0.4

    path_file = os.listdir(input_folder_root)

    
    for pth_file in path_file:
        input_folder = os.path.join(input_folder_root, pth_file)
        output_folder = os.path.join(output_folder_root, "ood_"+ pth_file + "_"+ str(patch_size)) 

        # 确保输出文件夹存在
        os.makedirs(output_folder, exist_ok=True)
        precess_image_two(input_folder, output_folder, image_size=image_size, patch_size=patch_size, swap_ratio=swap_ratio)
    print("Finish!")

if __name__ == "__main__":
    main()


## create pesudo OOD data from two files

In [ ]:

import os
import torch
import torchvision.transforms as transforms
from PIL import Image
import random
import torchvision

def split_image_to_patches(image, patch_size=64):
    _, height, width = image.size()
    num_patches_h = height // patch_size
    num_patches_w = width // patch_size
    patches = []
    for h in range(num_patches_h):
        for w in range(num_patches_w):
            start_h, start_w = h * patch_size, w * patch_size
            end_h, end_w = start_h + patch_size, start_w + patch_size
            patch = image[:, start_h:end_h, start_w:end_w]
            patches.append(patch)
    return patches

def shuffle_patches(patches):
    shuffled_patches = patches.copy()
    random.shuffle(shuffled_patches)
    return shuffled_patches

def combine_patches(patches, image_size):
    num_patches = len(patches)
    num_patches_per_row = image_size // patches[0].shape[1]
    combined_image = torch.cat([torch.cat(patches[i:i+num_patches_per_row], dim=2) for i in range(0, num_patches, num_patches_per_row)], dim=1)
    return combined_image

def swap_patches(image1, image2, swap_ratio=0.4):
    num_patches = len(image1)
    num_swaps = int(num_patches * swap_ratio)
    swap_indices = random.sample(range(num_patches), num_swaps)
    for idx in swap_indices:
        image1[idx], image2[idx] = image2[idx], image1[idx]
    return image1, image2



def precess_image_two_class(input_folder, output_folder, image_size=256, patch_size=64, swap_ratio=0.4):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
    ])
    
    
    image_files_0 = [file for file in os.listdir(input_folder[0]) if file.endswith(".jpg") or file.endswith(".png") or file.endswith(".JPEG")]
    random.shuffle(image_files_0)
    # print(image_files)
    image_files_1 = [file for file in os.listdir(input_folder[1]) if file.endswith(".jpg") or file.endswith(".png") or file.endswith(".JPEG")]
    random.shuffle(image_files_1)
    # print(image_files)

    
 

    for idx in range(len(image_files_0)):
        img_path_0 = os.path.join(input_folder[0], image_files_0[idx])
        image_0 = Image.open(img_path_0).convert("RGB")
        image_0 = transform(image_0).unsqueeze(0)[0]
        patches_0 = split_image_to_patches(image_0, patch_size=patch_size)
        shuffled_patches_0 = shuffle_patches(patches_0)

        img_path_1 = os.path.join(input_folder[1], image_files_1[idx])
        image_1 = Image.open(img_path_1).convert("RGB")
        image_1 = transform(image_1).unsqueeze(0)[0]
        patches_1 = split_image_to_patches(image_1, patch_size=patch_size)
        shuffled_patches_1 = shuffle_patches(patches_1)

        shuffled_patches_0, shuffled_patches_1 = swap_patches(shuffled_patches_0, shuffled_patches_1, swap_ratio=swap_ratio)

        combined_image0 = combine_patches(shuffled_patches_0, image_size)
        combined_image1 = combine_patches(shuffled_patches_1, image_size)

        torchvision.utils.save_image(combined_image0, os.path.join(output_folder[0], "combined_" + image_files_0[idx].strip(".JPEG") + "_" + image_files_1[idx].strip(".JPEG") +"_image1.jpg"))
        torchvision.utils.save_image(combined_image1, os.path.join(output_folder[1], "combined_" + image_files_0[idx].strip(".JPEG") + "_" + image_files_1[idx].strip(".JPEG") +"_image2.jpg"))
   

def main():
    # 假设有一个包含图像数据的文件夹
    input_folder_root = "/data31/data31_3/Public/Datasets/imagenet_few/val_1"  # 输入文件夹路径
    output_folder_root = "/data31/data31_3/Public/Datasets/imagenet_few/output" # 输出文件夹路径
    image_size = 224
    patch_size = 32
    swap_ratio = 0.4

    path_file = os.listdir(input_folder_root)

    input_folder = []
    output_folder = []
    for idx_img_file in range(0, len(path_file), 2):
        input_folder.append(os.path.join(input_folder_root, path_file[idx_img_file]))
        input_folder.append(os.path.join(input_folder_root, path_file[idx_img_file+1]))

        # 确保输出文件夹存在
        os.makedirs(os.path.join(output_folder_root, "ood_"+ path_file[idx_img_file] + "_"+ str(patch_size)), exist_ok=True)
        os.makedirs(os.path.join(output_folder_root, "ood_"+ path_file[idx_img_file+1] + "_"+ str(patch_size)), exist_ok=True)

        output_folder.append(os.path.join(output_folder_root, "ood_"+ path_file[idx_img_file] + "_"+ str(patch_size)))
        output_folder.append(os.path.join(output_folder_root, "ood_"+ path_file[idx_img_file+1] + "_"+ str(patch_size)))
        precess_image_two_class(input_folder, output_folder, image_size=image_size, patch_size=patch_size, swap_ratio=swap_ratio)
        input_folder = []
        output_folder = []
    print("Finish!")

if __name__ == "__main__":
    main()
